<a href="https://colab.research.google.com/github/EvelyneKol/Chatbot-for-Disease-Prediction/blob/main/SBERT_TRIALS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import os
# Import the library to mount Google Drive
from google.colab import drive

# Mount the Google Drive at /content/drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/thesis/"

# Set working directory to your Google Drive path
os.chdir('/content/drive/My Drive/thesis/')

# symptoms of my dataset
data=pd.read_csv("Training.csv")
list(data.columns.values)
list(data)
symptom_list = list(data)
print(symptom_list)

Mounted at /content/drive
app.py	    symptom_embeddings1.npy    Training.csv
knn.joblib  symptom_embeddings_qa.npy
['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing', 'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting', 'vomiting', 'burning_micturition', 'spotting_ urination', 'fatigue', 'weight_gain', 'anxiety', 'cold_hands_and_feets', 'mood_swings', 'weight_loss', 'restlessness', 'lethargy', 'patches_in_throat', 'irregular_sugar_level', 'cough', 'high_fever', 'sunken_eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish_skin', 'dark_urine', 'nausea', 'loss_of_appetite', 'pain_behind_the_eyes', 'back_pain', 'constipation', 'abdominal_pain', 'diarrhoea', 'mild_fever', 'yellow_urine', 'yellowing_of_eyes', 'acute_liver_failure', 'fluid_overload', 'swelling_of_stomach', 'swelled_lymph_nodes', 'malaise', 'blurred_and_distorted_vision', 'phlegm', 'throat_irritation', 'redness_of_eyes', '

In [10]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# List of SBERT models to evaluate
sbert_models = [
    'all-mpnet-base-v2',
    'multi-qa-mpnet-base-dot-v1',
    'all-distilroberta-v1',
    'all-MiniLM-L12-v2',
    'multi-qa-distilbert-cos-v1',
    'all-MiniLM-L6-v2',
]

# Your input examples and corresponding ground truth symptoms
examples = [
     {"input": "I've been feeling incredibly exhausted lately, with a constant cough, high fever, and a pounding headache. My nose won't stop running, and it's completely stuffed. I’m also dealing with chest pain, sore muscles, and I’ve completely lost my sense of smell.",
      "expected": {'fatigue', 'cough', 'high_fever', 'headache',
                     'runny_nose', 'congestion', 'chest_pain',
                     'loss_of_smell', 'muscle_pain'}},
    {
        "input": "I can’t stop sneezing—it's just one after another. I’m shivering all the time, getting chills, and my eyes are constantly watering. It’s really uncomfortable.",
        "expected": {'continuous_sneezing', 'shivering', 'chills', 'watering_from_eyes'}
    },
    {
        "input": "I feel really tired and have a persistent cough with thick mucus. My fever is high, and I’m often out of breath. I know similar issues run in my family, so I’m a bit concerned.",
        "expected": {'fatigue', 'cough', 'high_fever', 'breathlessness', 'family_history', 'mucoid_sputum'}},
     {
      "input": "I’ve been vomiting and have had constant diarrhea for hours. My eyes look sunken, and I’m pretty sure I’m getting dehydrated.",
      "expected": {'vomiting', 'sunken_eyes', 'dehydration', 'diarrhoea'}
     },

      {
      "input": "I've had serious indigestion and acidity lately, along with a splitting headache. My vision gets blurry and distorted at times, and I’m feeling unusually hungry. My neck is stiff, and I’ve been really irritable and even a bit depressed lately.",
      "expected": {'acidity', 'indigestion', 'headache', 'blurred_and_distorted_vision', 'excessive_hunger', 'stiff_neck', 'depression', 'irritability', 'visual_disturbances'}
     }
]

# Helper function to clean up symptoms
def preprocess_symptoms(symptoms):
    return [symptom.replace("_", " ").lower() for symptom in symptoms]

# Map symptoms using SBERT
def map_symptoms(user_input, model, model_name):
    # Preprocess input
    processed_input = user_input.lower()

    # Encode user input and known symptoms
    input_embedding = model.encode([processed_input])
    normalized_symptoms = preprocess_symptoms(symptom_list)
    symptom_embeddings = model.encode(normalized_symptoms)

    # Compute cosine similarities
    similarities = util.pytorch_cos_sim(input_embedding, symptom_embeddings)

    # Match if above threshold
    threshold = 0.35
    matches = [
        (normalized_symptoms[i], similarities[0][i].item())
        for i in range(len(normalized_symptoms))
    ]
    matches = [symptom for symptom, score in matches if score > threshold]

    # Return as set
    return set(matches)

# Metric computation
def calculate_metrics(predicted, expected):
    tp = len(predicted & expected)
    fp = len(predicted - expected)
    fn = len(expected - predicted)

    accuracy = tp / len(expected) if expected else 0
    precision = tp / (tp + fp) if (tp + fp) else 0
    recall = tp / (tp + fn) if (tp + fn) else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0

    return accuracy, precision, recall, f1

# Main evaluation loop
final_results = []

for model_name in sbert_models:
    print(f"\nEvaluating model: {model_name}")
    model = SentenceTransformer(model_name)

    acc_total = 0
    prec_total = 0
    rec_total = 0
    f1_total = 0

    for example in examples:
        predicted = map_symptoms(example["input"], model, model_name)
        expected = example["expected"]

        acc, prec, rec, f1 = calculate_metrics(predicted, expected)

        acc_total += acc
        prec_total += prec
        rec_total += rec
        f1_total += f1

    num_examples = len(examples)
    final_results.append({
        "model": model_name,
        "accuracy": acc_total / num_examples,
        "precision": prec_total / num_examples,
        "recall": rec_total / num_examples,
        "f1_score": f1_total / num_examples
    })

# Sort by F1 score or accuracy
final_results.sort(key=lambda x: x['f1_score'], reverse=True)

# Display results
print("\nModel Comparison Results:")
for res in final_results:
    print(f"\nModel: {res['model']}")
    print(f"Average Accuracy: {res['accuracy']:.2%}")
    print(f"Average Precision: {res['precision']:.2%}")
    print(f"Average Recall: {res['recall']:.2%}")
    print(f"Average F1 Score: {res['f1_score']:.2%}")
    print("-" * 50)


Evaluating model: all-mpnet-base-v2

Evaluating model: multi-qa-mpnet-base-dot-v1

Evaluating model: all-distilroberta-v1

Evaluating model: all-MiniLM-L12-v2

Evaluating model: multi-qa-distilbert-cos-v1

Evaluating model: all-MiniLM-L6-v2

Model Comparison Results:

Model: multi-qa-distilbert-cos-v1
Average Accuracy: 31.11%
Average Precision: 28.67%
Average Recall: 31.11%
Average F1 Score: 27.74%
--------------------------------------------------

Model: all-MiniLM-L12-v2
Average Accuracy: 36.67%
Average Precision: 20.59%
Average Recall: 36.67%
Average F1 Score: 24.23%
--------------------------------------------------

Model: all-MiniLM-L6-v2
Average Accuracy: 30.00%
Average Precision: 19.18%
Average Recall: 30.00%
Average F1 Score: 21.35%
--------------------------------------------------

Model: all-mpnet-base-v2
Average Accuracy: 27.78%
Average Precision: 14.70%
Average Recall: 27.78%
Average F1 Score: 17.51%
--------------------------------------------------

Model: multi-qa-mp